In [1142]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [1143]:
# -- years being used for evaluation
Years = '2004'

# -- spatial key being used for calculation
SpatialKey = 'ZIPCODE'

# -- use Fiscal or Calendar year
UseFiscalYear = True

# -- input path
if UseFiscalYear:
    InputPath = '../../shapeData/Dataframes_Fiscal/'
else:
    InputPath = '../../shapeData/Dataframes/'

# -- Census Park Breakdown input file
CensusParkInputFile = '../../shapeData/Tables/Park_Area_Census_Tract.xlsx'

# -- output path
OutputPath = '../../shapeData/Tables/'
WriteOutputPath = '../../shapeData/Tables/ParkQualityScores/'

In [1144]:
# -- read the park quality file on a year basis 

# -- for each year of analysis, merge files into 1

# -- clear df first so block can be rerun in notebook
yearDF = pd.DataFrame({})
for Year in Years.split(','):
    print "reading %s_Inspections.csv..." % (Year)
    df = pd.read_csv(InputPath + '%s_Inspections.csv' % (Year), index_col=0, sep='\t')
    try:
        yearDF = yearDF.append(df, ignore_index=True)
    except:
        yearDF = df

# -- append resorts columns.  reinstate column order
yearDF = yearDF.reindex_axis(df.columns, axis=1)

reading 2013_Inspections.csv...
reading 2014_Inspections.csv...
reading 2015_Inspections.csv...


In [1145]:
# -- read the park listing breakdowns by census tract
if SpatialKey == 'GEOID':
    print "reading census park info file..."
    censusParkDF = pd.read_excel(CensusParkInputFile)

In [1146]:
# -- read the Amenity files 
print "reading Amenity Files"
amenityFlagFilter = pd.read_excel(OutputPath + "PIP_Inventory_CUSP.xlsx")
amenityCompleteList = pd.read_excel(OutputPath + "PIP_Inventory_Client.xlsx")

reading Amenity Files


In [1147]:
# -- categories
categories = [u'Athletic Fields', u'Benches',
       u'Fences', u'Glass', u'Graffiti', u'Horticultural Areas', u'Ice',
       u'Lawns', u'Litter', u'Paved Surfaces', u'Play Equipment',
       u'Safety Surface', u'Sidewalks', u'Trails', u'Trees', u'Water Bodies',
       u'Weeds']

In [1148]:
# -- calculate inspection scores [ratio]
yearDF['Ratings Ratio'] = 1 - yearDF[categories].mean(axis=1)

In [1149]:
# -- build attribute list to capture from inspection reports.  
# --   If Spatial included, grab it.  If not, its GEOID which is in a different file
yearDFAttr = ['Prop ID', 'PID_base', 'Category']
if SpatialKey in yearDF.columns:
    yearDFAttr.append(SpatialKey)

# -- seed final park stats DF
byParkID = yearDF[yearDFAttr].groupby(['Prop ID'], as_index = False)
parkStats = byParkID.first()


# -- average Park Score and Acres
byPark = yearDF.groupby(['Prop ID'], as_index = False)
parkAvgScore = byPark['Ratings Ratio','Adjusted Acres'].mean()
parkAvgScore.rename(columns={'Ratings Ratio':'Score'}, inplace=True)
parkAvgScore.rename(columns={'Adjusted Acres':'Acres'}, inplace=True)
parkStats = pd.merge(parkStats, parkAvgScore, on='Prop ID', how='left')

# -- Utilize new park table which has parks broken up by census tracts if used!
if SpatialKey == 'GEOID':
    parkStats = pd.merge(censusParkDF[['Prop ID', 'GEOID', 'Fractional Acres']], parkStats, on = 'Prop ID', how = 'left')
    
    # -- Replace acerage with census breakup acreage
    parkStats['Acres'] = parkStats['Fractional Acres']
    parkStats.drop('Fractional Acres', axis=1, inplace=True)

# -- calculate weighted Score for park
parkStats['Score * Acres'] = parkStats['Score'] * parkStats['Acres']

In [1150]:
# -- Build Amenity List Mask
AmenitiesTypesToInclude = amenityFlagFilter[amenityFlagFilter['Include'] == 1].set_index(['Feature', 'Type', 'Category'])

# -- Apply mask to get Park Amenities to sum
amenityCompleteList.set_index(['Feature', 'Type', 'Category'], inplace=True)
ParkAmenitiesToInclude = amenityCompleteList.loc[AmenitiesTypesToInclude.index].reset_index()
amenityCompleteList.reset_index(inplace=True)

# -- Sum per park
byPark = amenityCompleteList.groupby('Prop ID').size()

# -- Append quantity of amenities to Park reports
parksWithAmenities = parkStats.set_index('Prop ID')
parksWithAmenities['m'] = byPark
parksWithAmenities.reset_index(inplace = True)

# -- Replace Nulls with 0
parksWithAmenities['m'].fillna(0, inplace=True)

In [1151]:
# -- seed district_category stats DF to calculate the mean Ameneties in each zipcode
byCategory = parksWithAmenities.groupby([SpatialKey] + ['Category'], as_index = False)

# -- calculate average amneties in each zipcode <m>
catAmenity = byCategory['m'].mean()
catAmenity.rename(columns={'m':'<m>'}, inplace=True)

In [1152]:
# -- Join the amenity average table to the orginal park info table to calculate the equation
amenityEquationDataframe = pd.merge(parksWithAmenities, catAmenity, on=[SpatialKey,'Category'], how='left')

# -- Apply Equation
amenityEquationDataframe['m/<m>'] = amenityEquationDataframe['m']/amenityEquationDataframe['<m>']
amenityEquationDataframe['m/<m>'].replace(np.inf, 0)
amenityEquationDataframe['m/<m>'].fillna(0, inplace=True)
amenityEquationDataframe['1 + m/<m>'] = 1+(amenityEquationDataframe['m/<m>'])
amenityEquationDataframe['W'] = amenityEquationDataframe['Acres']*amenityEquationDataframe['1 + m/<m>']
amenityEquationDataframe['W*Q'] = amenityEquationDataframe['W']*amenityEquationDataframe['Score']

In [1153]:
# -- seed district_category stats DF
byCategory = amenityEquationDataframe.groupby([SpatialKey] + ['Category'], as_index = False)
categoryStats = byCategory.first()[[SpatialKey] + ['Category']]

# -- sum WQ and W per category
catSums = byCategory['Acres', 'Score', 'Score * Acres','W*Q', 'W'].sum()
# catSums.rename(columns={'Weighted Score':'Score'}, inplace=True)
categoryStats = pd.merge(categoryStats, catSums, on=[SpatialKey] + ['Category'], how='left')

# -- count parks per category
catCounts = byCategory.size().reset_index().rename(columns={0:'Counts'})
categoryStats = pd.merge(categoryStats, catCounts, on=[SpatialKey] + ['Category'], how='left')

In [1154]:
# -- group by zipcode and category
bySpatialKey = categoryStats.groupby(SpatialKey, as_index = False)
SpatialKeySums = bySpatialKey.sum()[[SpatialKey] + ['Counts']]

categoryStats = pd.merge(categoryStats, SpatialKeySums, on=SpatialKey, how = 'left', suffixes = ['_Cat', '_%s' % SpatialKey])

In [1155]:
# -- calculate weighted scores for SpatialKey

# -- Normalized score
categoryStats['Non-Weighted Score'] = categoryStats['Score'] / categoryStats['Counts_Cat']
categoryStats['Naive Normalized Score'] = categoryStats['Non-Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

# -- Area Normalized Score
categoryStats['Weighted Score'] = categoryStats['Score * Acres'] / categoryStats['Acres']
categoryStats['Area Normalized Score'] = categoryStats['Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

# -- Area and Amenities weighted score
categoryStats['Equation Score'] = categoryStats['W*Q'] / categoryStats['W']
categoryStats['Amenities & Area Normalized Score'] = categoryStats['Equation Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

In [1156]:
# -- sum Scores per Zipcode
bySpatialKey = categoryStats.groupby(SpatialKey)
SpatialStats = bySpatialKey \
                    .sum() \
                    .reset_index() \
                    [[SpatialKey] + ['Naive Normalized Score'] + ['Area Normalized Score'] + ['Amenities & Area Normalized Score']]

In [1157]:
# -- floor scores at 0 just in case U/S overtake ratios
SpatialStats.loc[SpatialStats['Amenities & Area Normalized Score'] < 0, 'Amenities & Area Normalized Score'] = 0
SpatialStats.loc[SpatialStats['Area Normalized Score'] < 0, 'Area Normalized Score'] = 0
SpatialStats.loc[SpatialStats['Naive Normalized Score'] < 0, 'Naive Normalized Score'] = 0

In [1158]:
# -- Write to csv
now = datetime.now()
if UseFiscalYear:
    YearType = 'FiscalYr'
else:
    YearType = 'CalendarYr'
SpatialStats.to_csv(WriteOutputPath + 'ParkQuality_%s_%s_%s_%s.csv' % (SpatialKey, Years.replace(',','&'), YearType, now.strftime('%m-%d-%Y')), sep=',')

Done
